In [64]:
import mne
mne.set_log_level("CRITICAL")
import numpy as np
import double_dipper
from double_dipper import dataset, constants, io, ml
from double_dipper.constants import problem, strategy_prompt

from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras
np.random.seed(0)

### Helper Functions

In [39]:
def flatten_end(array):
    return array.reshape([array.shape[0], np.prod(array.shape[1:])])

In [40]:
def test_boundary(Y, preds, boundary = 0.5):
    rounded_preds = np.zeros_like(preds)
    rounded_preds[np.argwhere(preds >= boundary)] = 1
    (prec, rec, f1, _) = precision_recall_fscore_support(Y, rounded_preds, average="binary")
    print(f"precision = {prec}, recall = {rec}, f1 = {f1}")

In [41]:
def psds(trX, tsX):
    epochs = mne.EpochsArray(np.concatenate([trX,tsX],axis=0), constants.info)
    (psd_vals, _) = mne.time_frequency.psd_welch(epochs,
                                             fmax=42,
                                             tmin=problem.delay,tmax=strategy_prompt.delay)
    trPsd = psd_vals[:len(trX)]
    tsPsd = psd_vals[len(trX):]
    return (trPsd, tsPsd)

## IO

In [42]:
def labeller(meta):
    strat = meta["strategy"]
    if strat is None: return None
    if strat.lower().startswith("fact"):        return 0
    elif strat.lower().startswith("procedure"): return 1
    else:                                       return None

In [43]:
divider = lambda meta: (meta["id"], meta["epoch"])
subjNos = [i for i in range(1,11+1) if i != 5]
pairs = io.filePairs(*[f"cleaned/main/{i}" for i in subjNos])
dset = io.partition(divider, labeller, pairs)

In [61]:
def sk_predictor(trX, trY, tsX, model):
    trX = flatten_end(trX)
    tsX = flatten_end(tsX)
    model.fit(trX, trY)
    return model.predict(tsX)

## Partitioning

In [44]:
def preprocess(subjNo, split = .7):
    subjSet = dataset.subset(lambda k: k[0] == subjNo, dset)
    keys = sorted(subjSet.keys(), key= lambda k: k[1])
    X = np.concatenate([subjSet[k]["x"] for k in keys], axis = 0)
    Y = np.concatenate([subjSet[k]["y"] for k in keys], axis = 0)
    split_ind = int(len(X) * split)
    (trainX, testX) = (X[:split_ind], X[split_ind:])
    (trainY, testY) = (Y[:split_ind], Y[split_ind:])
    return (trainX, trainY, testX, testY)

In [45]:
(trX, trY, tsX, tsY) = preprocess(10)

## Temporal and Spectral

In [29]:
(trPsd, tsPsd) = psds(trX, tsX)
trX = np.concatenate([trX, trPsd], axis=-1)
tsX = np.concatenate([tsX, tsPsd], axis=-1)

#### Feed-forward

In [30]:
trX = trX.reshape([trX.shape[0], np.prod(trX.shape[1:])])
tsX = tsX.reshape([tsX.shape[0], np.prod(tsX.shape[1:])])

In [50]:
def deep_predictor(trainX, trainY, testX, kernel_size=128):
    model = keras.Sequential([
        keras.layers.Dense(kernel_size, input_shape=(trainX.shape[-1],), activation="relu"),
        keras.layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam",loss="binary_crossentropy")
    model.fit(trainX, trainY, steps_per_epoch=1, epochs=1000, batch_size=len(trainX))
    return model.predict(testX)

In [32]:
preds = deep_predictor(trX, trY, tsX)

Epoch 1/1000
1/1 [==============================] - 0s 214ms/step - loss: 0.6933
Epoch 2/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.6874
Epoch 3/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.6820
Epoch 4/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6774
Epoch 5/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6736
Epoch 6/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.6707
Epoch 7/1000
1/1 [==============================] - 0s 37ms/step - loss: 0.6687
Epoch 8/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6676
Epoch 9/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6672
Epoch 10/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.6675
Epoch 11/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.6681
Epoch 12/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.6688
Epoch 13/1000
1/1 [=================

Epoch 103/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6662
Epoch 104/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.6661
Epoch 105/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6661
Epoch 106/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6661
Epoch 107/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6661
Epoch 108/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.6661
Epoch 109/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.6661
Epoch 110/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.6660
Epoch 111/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6660
Epoch 112/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6660
Epoch 113/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.6660
Epoch 114/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6660
Epoch 115/1000
1

Epoch 203/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6631
Epoch 204/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6631
Epoch 205/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.6630
Epoch 206/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6630
Epoch 207/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6629
Epoch 208/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.6629
Epoch 209/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.6628
Epoch 210/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6627
Epoch 211/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6627
Epoch 212/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6626
Epoch 213/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.6626
Epoch 214/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6625
Epoch 215/1000
1

Epoch 303/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6515
Epoch 304/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6513
Epoch 305/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.6511
Epoch 306/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6509
Epoch 307/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.6507
Epoch 308/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6504
Epoch 309/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.6502
Epoch 310/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.6500
Epoch 311/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.6497
Epoch 312/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6495
Epoch 313/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.6493
Epoch 314/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6490
Epoch 315/1000
1

Epoch 403/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.6164
Epoch 404/1000
1/1 [==============================] - 0s 38ms/step - loss: 0.6159
Epoch 405/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6155
Epoch 406/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6150
Epoch 407/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6146
Epoch 408/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.6142
Epoch 409/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6138
Epoch 410/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6132
Epoch 411/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.6128
Epoch 412/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.6123
Epoch 413/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6119
Epoch 414/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.6115
Epoch 415/1000
1

Epoch 503/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.5727
Epoch 504/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.5723
Epoch 505/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.5718
Epoch 506/1000
1/1 [==============================] - 0s 31ms/step - loss: 0.5714
Epoch 507/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.5710
Epoch 508/1000
1/1 [==============================] - 0s 34ms/step - loss: 0.5705
Epoch 509/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.5701
Epoch 510/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.5696
Epoch 511/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.5692
Epoch 512/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.5687
Epoch 513/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.5683
Epoch 514/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.5678
Epoch 515/1000
1

Epoch 603/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.5259
Epoch 604/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.5256
Epoch 605/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.5251
Epoch 606/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.5245
Epoch 607/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.5239
Epoch 608/1000
1/1 [==============================] - 0s 39ms/step - loss: 0.5232
Epoch 609/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.5233
Epoch 610/1000
1/1 [==============================] - 0s 40ms/step - loss: 0.5223
Epoch 611/1000
1/1 [==============================] - 0s 47ms/step - loss: 0.5220
Epoch 612/1000
1/1 [==============================] - 0s 41ms/step - loss: 0.5217
Epoch 613/1000
1/1 [==============================] - 0s 43ms/step - loss: 0.5214
Epoch 614/1000
1/1 [==============================] - 0s 36ms/step - loss: 0.5208
Epoch 615/1000
1

Epoch 703/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.4747
Epoch 704/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.4742
Epoch 705/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.4736
Epoch 706/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.4732
Epoch 707/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4726
Epoch 708/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.4721
Epoch 709/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4716
Epoch 710/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.4711
Epoch 711/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.4706
Epoch 712/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4701
Epoch 713/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4696
Epoch 714/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4690
Epoch 715/1000
1

Epoch 803/1000
1/1 [==============================] - 0s 32ms/step - loss: 0.4261
Epoch 804/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.4255
Epoch 805/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.4249
Epoch 806/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4244
Epoch 807/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4239
Epoch 808/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4235
Epoch 809/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.4231
Epoch 810/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.4226
Epoch 811/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.4222
Epoch 812/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.4218
Epoch 813/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.4213
Epoch 814/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.4209
Epoch 815/1000
1

Epoch 903/1000
1/1 [==============================] - 0s 47ms/step - loss: 0.3831
Epoch 904/1000
1/1 [==============================] - 0s 37ms/step - loss: 0.3826
Epoch 905/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.3822
Epoch 906/1000
1/1 [==============================] - 0s 35ms/step - loss: 0.3816
Epoch 907/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.3811
Epoch 908/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.3809
Epoch 909/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.3806
Epoch 910/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.3803
Epoch 911/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.3797
Epoch 912/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.3793
Epoch 913/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.3789
Epoch 914/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.3784
Epoch 915/1000
1

In [33]:
test_boundary(tsY, preds, .5)

precision = 0.25, recall = 0.08333333333333333, f1 = 0.125


#### LDA

In [34]:
preds = ml.lda_predictor(trX, trY, tsX)
test_boundary(tsY, preds, .5)

precision = 0.3333333333333333, recall = 0.16666666666666666, f1 = 0.2222222222222222


/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


## Spectral Only

#### Feed-Forward

In [52]:
preds = deep_predictor(flatten_end(trPsd), trY, flatten_end(tsPsd), kernel_size=32)

Epoch 1/1000
1/1 [==============================] - 0s 182ms/step - loss: 0.6931
Epoch 2/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.6928
Epoch 3/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6923
Epoch 4/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6919
Epoch 5/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6914
Epoch 6/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6910
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6906
Epoch 8/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6901
Epoch 9/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6897
Epoch 10/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6893
Epoch 11/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6889
Epoch 12/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6884
Epoch 13/1000
1/1 [============================

Epoch 104/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6675
Epoch 105/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.6674
Epoch 106/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6674
Epoch 107/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6674
Epoch 108/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6674
Epoch 109/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6674
Epoch 110/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6674
Epoch 111/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.6673
Epoch 112/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6673
Epoch 113/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6673
Epoch 114/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6673
Epoch 115/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6673
Epoch 116/1000
1/1 [========

Epoch 206/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 207/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 208/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 209/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 210/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 211/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 212/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 213/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.6672
Epoch 214/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 215/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 216/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 217/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.6672
Epoch 218/1000
1/1 [========

1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 308/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 309/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 310/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 311/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 312/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 313/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 314/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 315/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 316/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 317/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 318/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 319/1000
1/1 [=======================

Epoch 409/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 410/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 411/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 412/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.6672
Epoch 413/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 414/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 415/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.6672
Epoch 416/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 417/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 418/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 419/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 420/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 421/1000
1/1 [========

1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 511/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 512/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 513/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 514/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 515/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 516/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 517/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.6672
Epoch 518/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 519/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 520/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 521/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 522/1000
1/1 [=======================

Epoch 612/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 613/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 614/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.6672
Epoch 615/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6672
Epoch 616/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.6672
Epoch 617/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 618/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6672
Epoch 619/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 620/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.6672
Epoch 621/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 622/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.6672
Epoch 623/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 624/1000
1/1 [========

Epoch 714/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 715/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 716/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 717/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 718/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.6672
Epoch 719/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 720/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 721/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 722/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.6672
Epoch 723/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 724/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 725/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 726/1000
1/1 [========

1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 816/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.6672
Epoch 817/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 818/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 819/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.6672
Epoch 820/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 821/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 822/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 823/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 824/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 825/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 826/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 827/1000
1/1 [======================

Epoch 917/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 918/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.6672
Epoch 919/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 920/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.6672
Epoch 921/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 922/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 923/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 924/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 925/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.6672
Epoch 926/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6672
Epoch 927/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.6672
Epoch 928/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 929/1000
1/1 [========

In [48]:
test_boundary(tsY, preds)

precision = 0.0, recall = 0.0, f1 = 0.0


/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


#### LDA

In [37]:
preds = ml.lda_predictor(trX, trY, tsX)
test_boundary(tsY, preds)

precision = 0.35714285714285715, recall = 0.4166666666666667, f1 = 0.3846153846153846


/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [19]:
recall

0.4166666666666667

#### Decision Tree

In [62]:
preds = sk_predictor(trX, trY, tsX, DecisionTreeClassifier())

In [63]:
test_boundary(tsY, preds)

precision = 0.5714285714285714, recall = 0.3333333333333333, f1 = 0.4210526315789474


In [68]:
preds = sk_predictor(trX, trY, tsX, RandomForestRegressor(n_estimators=100))

In [69]:
preds

array([0.62, 0.18, 0.3 , 0.52, 0.47, 0.39, 0.34, 0.24, 0.21, 0.47, 0.46,
       0.28, 0.45, 0.21, 0.37, 0.45, 0.51, 0.24, 0.29, 0.21, 0.59, 0.34,
       0.43, 0.14, 0.5 , 0.26, 0.36, 0.4 , 0.5 , 0.17, 0.25, 0.41, 0.4 ])

In [70]:
test_boundary(tsY, preds)

precision = 0.5, recall = 0.25, f1 = 0.3333333333333333
